# ESPIn Team Project - Flood Group
#### Ekta Aggarwal$^1$, Leia Barnes$^2$, Morgan Carrington$^3$, Prakash Pokhrel$^4$, Caroline Mierzejewski$^5$

[1] Univeristy, [2] University, [3] University, [4] University, [5] Texas State University, TX USA

## Introduction
This project shows how overtime the intensity and frequency of rain events can cause incision and lateral changes in stream channels.

Precipitation erodes landscapes creating new channels and causing others to migrate across floodplanes. When discharge increases in the channel due to more rainfall, sediment will easily be eroded and transported downstream until flow velocity decreases and sediment will eventually get deposited. 

# Initial Setup 
We neeed to first install necessary python packages and create the grid/ landscape for the project using ***.

### Importing necessary libraries and components

In [ ]:
# import packages

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

from landlab import RasterModelGrid
from landlab.components import FastscapeEroder, PriorityFloodFlowRouter
from landlab.plot import imshow_grid
from landlab.plot.drainage_plot import drainage_plot

import copy

### Define plotting functions and generate a grid

A raster model grid is intially created using rastermodelgrid showing a slanted bowl like landscape which we incorporated small hills scattered around the surface... Change Morgan or other topo?

In [ ]:
""" This part will change with Morgans topo code """

def surf_plot(mg, surface="topographic__elevation", title="Surface plot of topography"):
    plt.figure()
    ax = plt.axes(projection="3d")
    
    # Plot the surface.
    Z = mg.at_node[surface].reshape(mg.shape)
    color = cm.gray((Z - Z.min()) / (Z.max() - Z.min()))
    ax.plot_surface(
        mg.x_of_node.reshape(mg.shape),
        mg.y_of_node.reshape(mg.shape),
        Z,
        rstride=1,
        cstride=1,
        facecolors=color,
        linewidth=0.0,
        antialiased=False,
    )
    ax.view_init(elev=20, azim=-150)
    ax.set_xlabel("X axis")
    ax.set_ylabel("Y axis")
    ax.set_zlabel("Elevation")
    plt.title(title)
    plt.show()

# Create a raster model grid 

mg = RasterModelGrid((25, 30))
x, y = mg.x_of_node, mg.y_of_node
base_elevation =1.0 + x**2.2 + y**2
mg.add_field("topographic__elevation", base_elevation, at="node")

# Selecting nodes to influence to generate hills on topographic slope

num_bumps = 10 #picks 10 random number of nodes that will be raised up
bump_height = 300 #raises the randomly selected nodes 300
bump_radius = 4  # choses radius around each raised node to also effect
sigma = bump_radius / 2.0

# raised_nodes = np.random.choice(mg.number_of_nodes, size=num_bumps, replace=False) # Use this code to initially choose a number of nodes to raise on the hillslope 
# print("Random node indices:", raised_nodes) # If you would like to see which nodes are affected from the previous line, print the nodes that are raised

raised_nodes = np.array([197, 124, 494, 443,  60, 233, 627, 296, 600, 274]) # This code selects 10 nodes to raise that were initially chosen at random

# Add a Gaussian bump around each raised node

for node in raised_nodes:
    xc, yc = x[node], y[node]
    distances = np.sqrt((x - xc)**2 + (y - yc)**2)
    mask = distances < bump_radius
    bump = bump_height * np.exp(-((distances[mask])**2) / (2 * sigma**2))
    mg.at_node["topographic__elevation"][mask] += bump

# Plot the final terrain

surf_plot(mg, title="Watershed Topograhpy") #creates a 3d version of the topographic elevation with elevation ranging from ~ 2,000 to 0 

mg.imshow("topographic__elevation") #creates a 2d version of the topographic elevation

### Initialize flow router and plot drainage plot  
PriorityFloodFlowRouter is a component used to accumulate flow and calculate drainage area using the priorty flood algorithm. Perimeter nodes do not add to flux accumulation. 

In [ ]:
# set up parameters for the PriorityFloodFlowRouter

"""
Parameters: 
mg = raster model grid created above
surface = surface to direct flow across 
update_flow_depressions =  build-in depression handler
depresion_handler = breach or fill, breach creates drainage path on depression cell
"""

fa = PriorityFloodFlowRouter(
    mg, 
    surface="topographic__elevation",
    flow_metric="D8", #
    update_flow_depressions=True,
    depression_handler="breach", 
 )

In [ ]:
# run one step with PriorityFloodFlowRouter and create two plots

fa.run_one_step()
plt.figure()
drainage_plot(mg) # plot flow over topographic elevation
drainage_plot(mg, "drainage_area") #plot flow over drainage area 

### Initialize rain
This will be changed with Leia and Ekta's stuff

In [ ]:
"""
Parameters
?
?
?
"""

rain = 10.0 + 20.0 * np.random.rand(mg.number_of_nodes)
filtration=np.random.rand(mg.number_of_nodes)
final_flux_in=rain*(1-filtration)
#final_flux_in=0.8*rain

plt.imshow(rain.reshape(mg.shape), origin="lower", cmap="PuBu", vmin=0)
plt.colorbar()
plt.show()

_ = mg.add_field("water__unit_flux_in", final_flux_in, at="node", clobber=True)

### Rerun drainage with rain
After rainfall is added on the surface, a surface water discharge plot is recreated

In [ ]:
fa.run_one_step()
plt.figure()
drainage_plot(mg, "surface_water__discharge", title="Discharge")

### Something

In [ ]:
# stream power eroder component

"""
Fastscape Eroder Model Parameters: 
- mg: RasterModelGrid
- K_sp = constant that incorporates erodibility of rock efficiency of sed transport, and climate
- m_sp = exponent on drainage area
- n_sp = exponent on slope
- threshold_sp = erosion threshold
- discharge_field = [L^2/T]. The default is to use the grid field 'drainage_area', use "surfacce_water__discharge" for keyword argument
"""

sp = FastscapeEroder(
    mg,
    K_sp=0.0001, 
    m_sp=0.5,
    n_sp=1.0, 
    threshold_sp=0.0,
    discharge_field="surface_water__discharge",
)

# Run model for 1,000 years plotting every 100 years
run_duration = 1000.0  # duration of run, yr
dt = 10.0  # time-step duration, yr
plot_every = 100.0  # time interval for plotting, yr

# Derived parameters
nsteps = int(run_duration / dt)
next_plot = plot_every

# Set up colormap
cmap = copy.copy(mpl.colormaps["pink"])

for i in range(1, nsteps + 1):
    fa.run_one_step() # Priority Flood Flow Router
    sp.run_one_step(dt) # Fastscape Eroder
    
    if i * dt >= next_plot: # plot grids
        plt.figure()
        imshow_grid(
            mg,
            "topographic__elevation",
            grid_units=["m", "m"],
            var_name="Elevation (m)",
            cmap=cmap,
        )
        next_plot += plot_every

### What happens when we change the precipitation? 
Show code on this below